In [1]:
from gatubelxs.page_views import generate_page_view
from gatubelxs.click_events import generate_click_event
from gatubelxs.user_interaction import generate_user_interaction
from kafka import KafkaProducer
from gatubelxs.producer_monitor import ProducerMonitor
from random import randint
import json
import time

In [16]:
KAFKA_SERVER = '963ee95a1f99:9093'
TOPIC_PAGE_VIEWS = "page_views"
TOPIC_CLICK_EVENTS = "click_events"
TOPIC_USER_INTERACTIONS = "user_interactions"

NUMBER_SESSION = 400 #Number of web sessions to simulate
MAX_EVENTS_PER_SESSION = 10

In [17]:
#Producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVER,
    value_serializer=lambda m: json.dumps(m).encode('utf-8')
)

In [18]:
producer_monitor = ProducerMonitor(producer)

In [19]:
import uuid
from random import randint

def simulate_session():
    user_id = str(uuid.uuid4())
    session_id = str(uuid.uuid4())

    n_page_views = randint(1, MAX_EVENTS_PER_SESSION)
    n_clicks = randint(0, MAX_EVENTS_PER_SESSION)
    n_interactions = randint(0, MAX_EVENTS_PER_SESSION)

    stages = ["browse", "cart", "checkout"]
    stage_treshholds = [0.6, 0.85]

    for _ in range(n_page_views):
        evt = generate_page_view(user_id, session_id)
        producer_monitor.send_with_monitoring(TOPIC_PAGE_VIEWS, evt)
        print(f"[{TOPIC_PAGE_VIEWS}] ->", evt)
    
    for i in range(n_clicks):
        progress = i / max(n_clicks, 1)
        if progress < stage_treshholds[0]:
            stage = "browse"
        elif progress < stage_treshholds[1]:
            stage = "cart"
        else:
            stage = "checkout"
        evt = generate_click_event(user_id, session_id, user_stage=stage)
        producer_monitor.send_with_monitoring(TOPIC_CLICK_EVENTS, evt)
        print(f"[{TOPIC_CLICK_EVENTS}] ->", evt)
    
    for i in range(n_interactions):
        evt = generate_user_interaction(user_id, session_id)
        producer_monitor.send_with_monitoring(TOPIC_USER_INTERACTIONS, evt)
        print(f"[{TOPIC_USER_INTERACTIONS}] ->", evt)

In [20]:
try:
    print(f"Starting to produce sessions...")
    for i in range(NUMBER_SESSION):
        simulate_session()
        time.sleep(2)
except KeyboardInterrupt:
    print("Interrumpted by user; stopping.")
finally:
    producer.flush()
    producer.close()

Starting to produce sessions...
[page_views] -> {'user_id': 'd1d9cf4d-2953-43c4-b9d6-ccee1466ac4f', 'session_id': '9957b439-9acb-41d6-a639-4e9d4af76901', 'page_url': '/product/elect_1002', 'referrer_url': 'blog', 'category': 'electronics', 'price': 202.73, 'timestamp': '2025-05-13T03:35:08.321340'}
[page_views] -> {'user_id': 'd1d9cf4d-2953-43c4-b9d6-ccee1466ac4f', 'session_id': '9957b439-9acb-41d6-a639-4e9d4af76901', 'page_url': '/product/elect_1002', 'referrer_url': 'blog/blog', 'category': 'electronics', 'price': 202.73, 'timestamp': '2025-05-13T03:35:08.347694'}
[page_views] -> {'user_id': 'd1d9cf4d-2953-43c4-b9d6-ccee1466ac4f', 'session_id': '9957b439-9acb-41d6-a639-4e9d4af76901', 'page_url': '/product/elect_1002', 'referrer_url': 'search/app', 'category': 'electronics', 'price': 202.73, 'timestamp': '2025-05-13T03:35:08.347841'}
[page_views] -> {'user_id': 'd1d9cf4d-2953-43c4-b9d6-ccee1466ac4f', 'session_id': '9957b439-9acb-41d6-a639-4e9d4af76901', 'page_url': '/product/elect_100

In [21]:
producer_monitor.print_final_summary()

AttributeError: 'ProducerMonitor' object has no attribute 'print_final_summary'